In [1]:
import os
import warnings
import pandas as pd

In [9]:
def loadRes(problem, sizes, methods, new=False):
    # init a list to store processed results
    results = []
    # loop through each size
    for s in sizes:
        for m in methods:
            # retrive metrics
            metrics = retriveData(problem, s, m, new)
            # append the results to the list
            results.append(metrics)
    # convert into a Pandas DataFrame
    result_df = pd.DataFrame(results)
    # reshape
    result_df = reshapeDataframe(result_df, sizes)
    return result_df

def retriveData(problem, size, method, new):
    # method map
    method_mapping = {
    "cls": "RC", "thd": "LT", "exact": "EX", 
    "rel": "RR", "root": "N1", "lrn": "RL", "ste": "RS",
    "cls100": "RC", "thd100": "LT", "lrn100": "RL", "ste100": "RS",
    "cls1000": "RC", "thd1000": "LT", "lrn1000": "RL", "ste1000": "RS",
    }
    # dir and file
    data_path = "./result/"
    if problem == "rb":
        if new:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}_new.csv")
        else:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}.csv")
    else:
        if new:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}-{size}_new.csv")
        else:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}-{size}.csv")
    print(csv_file)
    if not os.path.exists(csv_file):
        # output a warning
        warnings.warn(f"File not found: {csv_file}")
        # return None values for all metrics
        return {
            "Method": method_mapping[method],
            "Problem Size": size,
            "Obj Mean": None,
            "Obj Median": None,
            "% Infeasible": None,
            "% Unsolved": None,
            "Time (Sec)": None,
            "Viol Mean": None,
            "Viol Max": None,
            "Num Viol Mean": None
        }
    # load data
    df = pd.read_csv(csv_file)
    # calculate the required metrics
    obj_mean = df["Obj Val"].mean()  # mean of the objective values
    obj_median = df["Obj Val"].median()  # median of the objective values
    percent_infeasible = (df["Num Violations"] > 0).mean() * 100  # percentage of infeasible instances
    mean_viols = df["Mean Violation"].mean()  # average mean violation
    max_viols = df["Max Violation"].max()  # maximum violation
    num_viols = df["Num Violations"].mean()  # average number of violations
    percent_unsolved = df["Obj Val"].isna().mean() * 100  # percentage of unsolved cases
    time_mean = df["Elapsed Time"].mean()  # Mean of the elapsed solving times
    # append the results to the list
    metrics = {
        "Method": method_mapping[method],
        "Problem Size": size,
        "Obj Mean": obj_mean,
        "Obj Median": obj_median,
        "% Infeasible": percent_infeasible,
        "% Unsolved": percent_unsolved,
        "Time (Sec)": time_mean,
        "Viol Mean": mean_viols,
        "Viol Max": max_viols,
        "Num Viol Mean": num_viols
    }
    return metrics

def reshapeDataframe(df, sizes):
    # define the metrics to include
    metrics = ["Obj Mean", "Obj Median", "% Infeasible", "% Unsolved", "Time (Sec)", "Viol Mean", "Viol Max", "Num Viol Mean"]
    # define the method to include
    methods = ["RC", "LT", "EX", "RR", "N1", "RL", "RS"]
    # reshape
    reshaped_data = []
    for method in methods:
        for metric in metrics:
            row = {"Method": method, "Metric": metric}
            for s in sizes:
                value = df[(df["Method"] == method) & (df["Problem Size"] == s)][metric].values
                # check if the value has exactly one element
                if len(value) != 1:
                    raise ValueError(f"Unexpected number of values ({len(value)}) for Method: {method}, Metric: {metric}, Problem Size: {s}")
                row[f"{s}×{s}"] = value[0]
            reshaped_data.append(row)
    # convert to a DataFrame
    reshaped_df = pd.DataFrame(reshaped_data)
    return reshaped_df

## Convex Quadratic

In [3]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls", "thd", "exact", "rel", "root", "lrn", "ste"]
loadRes(problem="cq", sizes=sizes, methods=methods)

,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.620135,-1.595961e+00,-4.237328e+00,-1.219593e+01,-1.354054e+01,-31.617419,-73.312205,-142.675594
1,RC,Obj Median,0.428045,-1.891001e+00,-4.307350e+00,-1.219962e+01,-1.359898e+01,-31.706233,-73.376950,-142.726241
2,RC,% Infeasible,4.000000,8.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,3.000000,14.000000,18.000000
3,RC,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001853,1.865625e-03,1.856163e-03,1.868107e-03,2.159805e-03,0.002132,0.002530,0.004238
5,RC,Viol Mean,0.000087,1.405149e-04,4.011002e-05,1.864139e-05,2.996753e-05,0.000011,0.000026,0.000040
6,RC,Viol Max,0.026017,4.184283e-02,8.022005e-02,9.320694e-02,9.049125e-02,0.095229,0.224482,0.678663
7,RC,Num Viol Mean,0.040000,8.000000e-02,1.000000e-02,1.000000e-02,4.000000e-02,0.040000,0.140000,0.190000
8,LT,Obj Mean,0.665490,-1.692566e+00,-4.302138e+00,-1.297686e+01,-1.365074e+01,-31.341756,-72.358103,-142.562616
9,LT,Obj Median,0.467924,-2.014438e+00,-4.319065e+00,-1.302667e+01,-1.377216e+01,-31.613921,-72.483963,-142.552094


### Simple Non-Convex

In [4]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls", "thd", "exact", "rel", "root", "lrn", "ste"]
loadRes(problem="nc", sizes=sizes, methods=methods)

C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_exact_50-50.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_exact_100-100.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_root_100-100.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_exact_200-200.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_root_200-200.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_exact_500-500.csv
  warnings.wa

,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.162234,0.824642,0.323959,0.788801,1.052989,0.572105,-0.573593,0.433432
1,RC,Obj Median,0.136189,0.484538,0.300903,0.765446,1.016585,0.550516,-0.574800,0.439652
2,RC,% Infeasible,2.000000,2.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000
3,RC,% Unsolved,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001863,0.002063,0.001876,0.001973,0.002126,0.002201,0.002695,0.005401
5,RC,Viol Mean,0.000034,0.000016,0.000036,0.000000,0.000000,0.000008,0.000004,0.000003
6,RC,Viol Max,0.014079,0.013261,0.071382,0.000000,0.000000,0.087052,0.138625,0.304790
7,RC,Num Viol Mean,0.020000,0.020000,0.010000,0.000000,0.000000,0.020000,0.020000,0.010000
8,LT,Obj Mean,0.200238,0.681972,0.004122,0.295509,1.214635,0.147947,-2.644251,-5.026568
9,LT,Obj Median,0.112477,0.415811,-0.010939,0.268040,1.139384,0.022795,-2.664689,-5.075384


### Simple Non-Convex (Perturbed A)

In [5]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls100", "thd100", "exact", "rel", "root", "lrn100", "ste100"]
loadRes(problem="nc", sizes=sizes, methods=methods, new=True)

C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_exact_20-20_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_rel_20-20_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_exact_50-50_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_rel_50-50_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_rel_100-100_new.csv
  warnings.warn(f"File not found: {csv_file}")
C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\1572043496.py:39: UserWarning: File not found: ./result/nc_root_100-100_new.csv


,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.382745,1.101563,0.227695,7.712910e-01,1.663565,1.471528,0.526173,1.422244
1,RC,Obj Median,0.239879,0.837723,0.216590,7.519165e-01,1.594214,1.436150,0.526173,0.809124
2,RC,% Infeasible,1.000000,3.000000,0.000000,2.000000e+00,0.000000,1.000000,4.000000,3.000000
3,RC,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001919,0.001936,0.001928,2.014108e-03,0.002193,0.002170,0.002871,0.003962
5,RC,Viol Mean,0.000026,0.000067,0.000000,1.513940e-05,0.000000,0.000003,0.000006,0.000034
6,RC,Viol Max,0.012821,0.036319,0.000000,6.728357e-02,0.000000,0.067195,0.135748,1.432488
7,RC,Num Viol Mean,0.010000,0.030000,0.000000,2.000000e-02,0.000000,0.010000,0.040000,0.050000
8,LT,Obj Mean,0.359111,0.909994,0.194875,5.796574e-01,0.668875,-0.356054,-1.373667,-3.743579
9,LT,Obj Median,0.225594,0.682508,0.175219,5.663435e-01,0.648688,-0.373353,-1.593921,-3.716251


### Rosenbrock

In [10]:
sizes = [1, 10, 100, 1000, 10000]
methods = ["cls100", "thd100", "exact", "rel", "root", "lrn100", "ste100"]
loadRes(problem="rb", sizes=sizes, methods=methods)

./result/rb_cls100_1.csv
./result/rb_thd100_1.csv
./result/rb_exact_1.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_9748\346861903.py:40: UserWarning: File not found: ./result/rb_thd100_1.csv
  warnings.warn(f"File not found: {csv_file}")


KeyError: 'Num Violations'